# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace/Udcty-DataEngineering/Project2
['/home/workspace/Udcty-DataEngineering/Project2/event_data/2018-11-30-events.csv', '/home/workspace/Udcty-DataEngineering/Project2/event_data/2018-11-23-events.csv', '/home/workspace/Udcty-DataEngineering/Project2/event_data/2018-11-22-events.csv', '/home/workspace/Udcty-DataEngineering/Project2/event_data/2018-11-29-events.csv', '/home/workspace/Udcty-DataEngineering/Project2/event_data/2018-11-11-events.csv', '/home/workspace/Udcty-DataEngineering/Project2/event_data/2018-11-14-events.csv', '/home/workspace/Udcty-DataEngineering/Project2/event_data/2018-11-20-events.csv', '/home/workspace/Udcty-DataEngineering/Project2/event_data/2018-11-15-events.csv', '/home/workspace/Udcty-DataEngineering/Project2/event_data/2018-11-05-events.csv', '/home/workspace/Udcty-DataEngineering/Project2/event_data/2018-11-28-events.csv', '/home/workspace/Udcty-DataEngineering/Project2/event_data/2018-11-25-events.csv', '/home/workspace/Udcty-DataEngineering/

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# Printing the total number of rows 
print(len(full_data_rows_list))


# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance to local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Creating the keyspace in Cassandra
try:
    session.execute("""
           CREATE KEYSPACE IF NOT EXISTS musicdatabase
           WITH REPLICATION =
           {'class'       : 'SimpleStrategy',
            'replication_factor' : 1}
            """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Setting KEYSPACE to the keyspace specified above
try:
    session.set_keyspace("musicdatabase")
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# Songs Table: 
## For extracting the artist, song title and song's length in the music app history that was heard during particular session Id and item in session

- Create table tb_song
- Primary key is made using partition key sessionId, and clustering key itemInSession for filtering based on the requirements
- Table Schema is:
    - session_id INT PKey
    - item_in_session INT Ckey
    - artist TEXT
    - song_title TEXT
    - song_length FLOAT

In [8]:
# Query for creating the table tb_song 
SongtbQuery = "CREATE TABLE IF NOT EXISTS tb_song"
SongtbQuery = f"{SongtbQuery} (session_id INT, item_in_session INT,artist TEXT, song_title TEXT,\
                                    song_length FLOAT ,PRIMARY KEY ((session_id), item_in_session))"

# Creating the table with above query
try:
    session.execute(SongtbQuery)
except Exception as e:
    print(e)
    
# Verification query for extracting the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

SelectQuery = """SELECT artist, song_title, song_length\
                 FROM tb_song\
                 WHERE session_id = %s\
                 AND item_in_session = %s\
                 """
                    

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERT statements into the `query` variable
        query = "INSERT INTO tb_song(session_id, item_in_session, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### SELECT to verify that the data have been inserted into each table

In [10]:
## SELECT statement to verify the data was entered into the table
try:
    rows = session.execute(SelectQuery, (338, 4))
except Exception as e:
    print(e)
    
for row in rows:
    print("Artist: "+row.artist, "\nSong Title: "+row.song_title, "\nSong Length: "+str(row.song_length))

Artist: Faithless 
Song Title: Music Matters (Mark Knight Dub) 
Song Length: 495.30731201171875


# Artist Table: 
## For extracting the name of artist, song (sorted by itemInSession) and user (first and last name) for a particular userid and sessionid

- Create table tb_artist
- Primary key is made using partition key user_id and session_id, and clustering key itemInSession for sorting the results based on the requirements
- Table Schema is:
    - user_id INT PKey
    - session_id INT PKey
    - item_in_session INT CKey
    - artist_name TEXT
    - song_title TEXT
    - user_first_name TEXT
    - user_last_name TEXT

In [11]:
# Query for creating the table tb_artist 
ArtisttbQuery = "CREATE TABLE IF NOT EXISTS tb_artist"
ArtisttbQuery = f"{ArtisttbQuery} (user_id INT, session_id INT, item_in_session INT, \
                                        artist_name TEXT, song_title TEXT, user_first_name TEXT, \
                                        user_last_name TEXT, PRIMARY KEY ((user_id, session_id), item_in_session))"

# Creating the table with above query
try:
    session.execute(ArtisttbQuery)
except Exception as e:
    print(e)
    
    
# Verification query for extracting the name of artist, song (sorted by itemInSession) and user (first and last name) \
## for userid = 10  and sessionid = 182

    
SelectQuery_Artist = """ SELECT artist_name, song_title, user_first_name, user_last_name FROM tb_artist
                    WHERE user_id = %s
                    AND session_id = %s 
                    ORDER BY item_in_session
                    """           

In [12]:
# Insert values in artist table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO tb_artist(user_id, session_id, item_in_session, artist_name, \
                                            song_title, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### SELECT statement to verify the data was entered into the table

In [13]:
## SELECT statement to verify the data was entered into the table
try:
    rows = session.execute(SelectQuery_Artist, (10, 182))
except Exception as e:
    print(e)
    
for row in rows:
    print("Artist: "+row.artist_name, "\nSong Title: "+row.song_title, "\nUser First Name: "+row.user_first_name, "\nUser Last Name: "+row.user_last_name)

Artist: Down To The Bone 
Song Title: Keep On Keepin' On 
User First Name: Sylvie 
User Last Name: Cruz
Artist: Three Drives 
Song Title: Greece 2000 
User First Name: Sylvie 
User Last Name: Cruz
Artist: Sebastien Tellier 
Song Title: Kilometer 
User First Name: Sylvie 
User Last Name: Cruz
Artist: Lonnie Gordon 
Song Title: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 
User First Name: Sylvie 
User Last Name: Cruz


# User Table: 
## For extracting the user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

- Create table tb_user
- Primary key is made using partition key song_title, and clustering key user_id
- Table Schema is:
    - song_title TEXT
    - user_id INT
    - user_first_name TEXT
    - user_last_name TEXT

In [14]:
# Query for creating the table tb_user 
UsertbQuery = "CREATE TABLE IF NOT EXISTS tb_user"
UsertbQuery = f"{UsertbQuery}(song_title TEXT, user_id INT, user_first_name TEXT, user_last_name TEXT, \
                                        PRIMARY KEY ((song_title), user_id))"

# Creating the table with above query
try:
    session.execute(UsertbQuery)
except Exception as e:
    print(e)                 
    
# Verification query for extracting the user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

SelectQuery_User = """ SELECT user_first_name, user_last_name FROM tb_user WHERE song_title = %s;"""


In [15]:
# Insert values in User table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO tb_user(song_title, user_id, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### SELECT statement to verify the data was entered into the table

In [16]:
## SELECT statement to verify the data was entered into the table
try:
    rows = session.execute(SelectQuery_User, ('All Hands Against His Own', ))
except Exception as e:
    print(e)

for row in rows:
    print("User First Name: "+row.user_first_name, "\nUser Last Name: "+row.user_last_name)

User First Name: Jacqueline 
User Last Name: Lynch
User First Name: Tegan 
User Last Name: Levine
User First Name: Sara 
User Last Name: Johnson


### Drop the tables before closing out the sessions

In [17]:
## Drop the table before closing out the sessions

In [18]:
session.execute("DROP TABLE IF EXISTS tb_song")
session.execute("DROP TABLE IF EXISTS tb_artist")
session.execute("DROP TABLE IF EXISTS tb_user")

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()